In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import torch

from itertools import chain

import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
import sys

sys.path.append('..')

import utils

from importlib import reload

reload(utils)

from utils import dataset, applier, domain, pipe

reload(dataset)
reload(applier)
reload(domain)
reload(pipe)

<module 'utils.pipe' from '../utils/pipe.py'>

In [4]:
data_path = "/Users/sergmiller/Documents/my/IDAO-2021/data/idao_dataset/"

In [5]:
train_path = os.path.join(data_path, "train")

In [6]:
sample_dataset = dataset.build_dataset(train_path, limit=10)

100%|██████████| 10/10 [00:00<00:00, 132.55it/s]


In [10]:
sample_dataset.save('test.npz')
sample_dataset = dataset.LabeledDataset.load('test.npz')

In [11]:
sample_dataset.samples.shape

(10, 576, 576)

In [50]:
train_dataset = build_particles_dataset(train_path)

100%|██████████| 13404/13404 [01:44<00:00, 128.30it/s]


In [12]:
import torchvision.models as models
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
model = models.mobilenet_v3_small(pretrained=True)

In [13]:
torch.set_num_threads(1)

In [14]:
torch.get_num_threads()

1

In [17]:
emb_dataset = applier.build_embd_dataset(sample_dataset)

100%|██████████| 1/1 [00:00<00:00, 5289.16it/s]


In [18]:
emb_dataset.samples.shape

(10, 1000)

In [20]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score

In [21]:
params = {
    "penalty": 'l1',
    "solver": 'saga',
    "verbose": 10,
    "max_iter": 100,
    "tol": 1e-5,
    "scoring" : 'roc_auc',
    "random_state": 0,
    "cv": 5,
    "n_jobs": 7
}

In [22]:
cv_label1 = LogisticRegressionCV(**params).fit(
    emb_dataset.samples, (emb_dataset.labels[:, 0] == 'NR').astype(int))

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [124]:
np.mean(cv_label1.scores_[1], axis=0)

array([0.5       , 0.5       , 0.7952889 , 0.87402804, 0.94469283,
       0.95739159, 0.96157609, 0.96381906, 0.96529288, 0.96639186])

In [122]:
cv_label1.Cs_

array([1.00000000e-04, 7.74263683e-04, 5.99484250e-03, 4.64158883e-02,
       3.59381366e-01, 2.78255940e+00, 2.15443469e+01, 1.66810054e+02,
       1.29154967e+03, 1.00000000e+04])

In [114]:
cv_label1.get_params()

{'Cs': 10,
 'class_weight': None,
 'cv': 3,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1.0,
 'l1_ratios': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l1',
 'random_state': 0,
 'refit': True,
 'scoring': 'roc_auc',
 'solver': 'saga',
 'tol': 1e-05,
 'verbose': 10}

In [23]:
import copy

classes = [1, 10, 20, 3, 30, 6]

def mae_scorer(estimator, X, y):
    y_pred = [classes[int(_y)] for _y in estimator.predict(X)]
    y_pred = np.array(y_pred)
    y = np.array([classes[_y] for _y in y])
    return np.mean(np.abs(y_pred - y))

params2 = copy.copy(params)

params2['scoring'] = mae_scorer
# params2['class_weight'] = {str(c) : c for c in classes}
cv_label2 = LogisticRegressionCV(**params2).fit(
    emb_dataset.samples, emb_dataset.labels[:, 1])

/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn

convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds


convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds


convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds


convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds

convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds

convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
max_iter reached after 0 seconds
max_iter reached after 0 seconds
max_iter reached after 0 seconds
max_iter reached after 0 secondsmax_iter reached after 0 seco

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 1 secondsmax_iter reached after 1 seconds
max_iter reached after 1 seconds

max_iter reached after 1 seconds
max_iter reached after 1 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

...

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 0 seconds
max_iter reached after 0 seconds
max_iter reached after 0 seconds
max_iter reached after 0 secondsmax_iter reached after 0 seconds

max_iter reached after 0 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

max_iter reached after 0 seconds
max_iter reached after 0 seconds
max_iter reached after 1 seconds
max_iter reached after 1 seconds
max_iter reached after 1 seconds
max_iter reached after 0 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [24]:
np.mean(cv_label2.scores_['1'],axis=0) # mini weighted

array([7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.7, 5.7, 5.7, 5.7])

In [142]:
np.mean(cv_label2.scores_['1'],axis=0) # not weighted

array([10.68017027, 10.68017027, 10.09729052,  9.80483741,  9.63690119,
        9.57527796,  9.54491318,  9.53208126,  9.52514296,  9.51149031])

In [144]:
np.mean(cv_label2.scores_['1'],axis=0) # weighted

array([ 9.59965186,  9.4570564 ,  9.96051226, 10.08321655,  9.88160343,
        9.84861552,  9.80512292,  9.85751204,  9.71321238,  9.77365697])

In [28]:
pub_test_dataset = dataset.build_dataset(
    os.path.join(data_path, "public_test"),
    sample_processor=domain.process_test_sample, limit=10)

100%|██████████| 10/10 [00:00<00:00, 97.71it/s]


In [29]:
private_test_dataset = dataset.build_dataset(
    os.path.join(data_path, "private_test"),
    sample_processor=domain.process_test_sample, limit=10)

100%|██████████| 10/10 [00:00<00:00, 95.26it/s]


In [32]:
sample_res = pipe.apply_all_models_to_test_dataset(
    sample_dataset,
    cv_label2,
    cv_label2,
    'sample')

100%|██████████| 1/1 [00:00<00:00, 4809.98it/s]


In [33]:
sample_res.tags.shape, sample_res.samples.shape

((10,), (10, 1000))

In [36]:
pub_test_predictions = pipe.apply_all_models_to_test_dataset(
    pub_test_dataset, cv_label2, cv_label2, 'pub_test')

100%|██████████| 1/1 [00:00<00:00, 1085.76it/s]


In [37]:
private_test_predictions = pipe.apply_all_models_to_test_dataset(
    private_test_dataset, cv_label2, cv_label2, 'private_test')

100%|██████████| 1/1 [00:00<00:00, 576.70it/s]


In [40]:
dataset.dataset2submit_csv(sample_dataset, "sample_dataset_sub.csv")

In [41]:
test_submit = dataset.LabeledDataset.merge(
    pub_test_predictions, private_test_predictions)

In [42]:
dataset.dataset2submit_csv(test_submit,"test.csv")

In [43]:
test_submit.tags.shape

(20,)

In [44]:
test_submit.samples.shape

(20, 1000)

In [45]:
pub_test_predictions.tags.shape, pub_test_predictions.samples.shape

((10,), (10, 1000))

In [46]:
private_test_predictions.tags.shape, private_test_predictions.samples.shape

((10,), (10, 1000))